In [2]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
linkage_name = '20190423_115116_meshID_24e48bbe-b793-446c-9bb2-afbe71f29f52'
l = elastic_rods.RodLinkage('../../examples/florin/{}.obj'.format(linkage_name), 8)
#driver=166
driver=l.centralJoint()

#mat = elastic_rods.RodMaterial('+', 2000, 0.3, [5, 5, 0.7, 0.7], stiffAxis=elastic_rods.StiffAxis.D1)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [12, 8])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
#view.setCameraParams(((2.656451698624758, 1.5087349593639046, 0.23168476156681694),
# (0.052583904614345374, 0.026224725204501566, 0.9982721055721517),
# (0.13680349418553736, -0.13680349418553736, -0.8425278443781967)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [5]:
l.saveVisualizationGeometry('deployed_{}.obj'.format(linkage_name))
#l.writeLinkageDebugData('')

In [14]:
from write_render_files import writeRenderFiles, writeActuators
writeRenderFiles(l, 'Pavilion12', 'pav12')
#writeActuators(l, np.where(torques > 1e-4)[0], directory='AsymPointyDataFlat', name='asymPointy_Fab')

In [10]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_{}.txt'.format(linkage_name), zeroBasedIndexing=True)
np.savetxt('restlen_{}.txt'.format(linkage_name),l.getPerSegmentRestLength())
np.savetxt('normals_{}.txt'.format(linkage_name), np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_{}.msh'.format(linkage_name))

In [41]:
# Compute maximum "stress" appearing at any rod vertex in the structure
max([max(segmentStress) for segmentStress in l.rodStresses()[1]])

24.37145054902055

In [ ]:
# Output actuation torque
for forceScale in np.linspace(1.0, 0, 51):
    name = 'Pavilion14_{}'.format(forceScale)
    with suppress_stdout(): elastic_rods.compute_equilibrium(l, forceScale * externalForces, fixedVars=fixedVars)
    view.update(preserveExisting=True)
    writeRenderFiles(l, directory, name)
    writeActuators(l, np.where(torques > 0.1)[0], directory, name)

In [44]:
# Compute average "stress" appearing over all rod vertices in the structure
np.mean(sum(l.rodStresses()[1], []))

11.81760497978405

In [4]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(100) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

target angle:  1.9163715186897714
0	635777	629.482	629.482	1	0
Backtracking failed.
1.8094204157719485	635768.5041055225	635768.5041055225	621981.6675376657	12760.977344712914	1025.859223143767
target angle:  1.9128808601857827
0	631810	626.004	626.004	1	0
Backtracking failed.
1.8049721405741115	631801.5076136177	631801.5076136177	618160.5390787007	12621.20291846221	1019.7656164545588
target angle:  1.909390201681794
0	627866	622.545	622.545	1	0
1.800529124156734	627856.9869823317	627856.9869823316	614360.2315223531	12483.065222372205	1013.6902376060158
target angle:  1.9058995431778054
0	623944	619.108	619.108	1	0
Backtracking failed.
1.796091369084057	623934.8568955248	623934.8568955247	610580.6701169155	12346.553580425367	1007.633198183982
target angle:  1.9024088846738167
0	620044	615.688	615.688	1	0
1.791658877919308	620035.032126145	620035.0321261451	606821.780285304	12211.657243002728	1001.5945978382384
target angle:  1.898918226169828
0	616166	612.294	612.294	1	0
1.787231653112